In [1]:
from qdrant_client import models,QdrantClient
import numpy
#pip install 'numpy<2'
import time
import threading
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy
import random
import datetime


e:\conda\envs\GEN_AI_T1\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def model_init():
    #tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER") #for a different NER 
    #model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER") #for a different NER
    ref_model = SentenceTransformer('all-MiniLM-L6-v2')
    nlp = spacy.load('en_core_web_sm')
    #nlp = pipeline("ner", model=model, tokenizer=tokenizer) #for a different NER
    return ref_model,nlp

def location_ner(nlp,res):
    doc = nlp(res)
    # Extract entities recognized as locations (GPE: Geopolitical Entities, LOC: Non-GPE locations)
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    if locations:
        return ', '.join(locations)
    else:
        return 0
    #"I didn't understand your response. Could you please provide the area that you are located in right now?"
    
def is_semantically_related(ref_model,sentence, reference_sentences, threshold=0.7):
    sentence_embedding = ref_model.encode(sentence, convert_to_tensor=True)
    for ref_sentence in reference_sentences:
        ref_embedding = ref_model.encode(ref_sentence, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(sentence_embedding, ref_embedding)
        if similarity.item() > threshold:
            return True, similarity.item()

    return False, 0

def relevance_check(ref_model,user_sentence): 
    reference_sentences_set1 = ["too late", "arrive too late","i am panicing", "won't make it in time", "delayed", "take too long", "not soon enough", "we can't wait", "need help immediately", "time-sensitive", "urgent" ,"The doctor will not arrive in time.", "I am worried the doctor will be too late."]
    reference_sentences_set2 = ["No Thank you",'Thanks for all the help','Thanks']
    is_related_1, similarity_score_1 = is_semantically_related(ref_model,user_sentence, reference_sentences=reference_sentences_set1)
    is_related_2, similarity_score_2 = is_semantically_related(ref_model,user_sentence, reference_sentences=reference_sentences_set2)
    if is_related_1 or is_related_2:
        if similarity_score_1 > similarity_score_2:
            return 1
        else:
            return 2
    else:
        return 0
    
def vector_search(qdrant,encoder,x,collect_name):
    # Search within a Vector database 
    #args:
    #qdrant: object of QdrantClient
    #encoder:  encoder for semantic similarity
    #x: sentence for calculating semantic similarity with records 
    hits = qdrant.search(
        collection_name=collect_name,
        query_vector=encoder.encode(x).tolist(),
        limit=1
    )
    for hit in hits:
        return (hit.payload,'score:',hit.score)


def process_response(res):
    # Generating the Output of vector search
    encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    qdrant=vector_database_init()
    llm_output = f"{vector_search(qdrant,encoder,res,"Responses")[0]["name"]}"
    return llm_output

def vector_database_init():
    # initializing Qdrant client at port 6333
    qdrant = QdrantClient(
        host='localhost',
        port=6333
    )
    return qdrant


def process(symptoms):
    # Generating the Output of vector search
    print("\nProcessing the emergency report...")
    time.sleep(15)  # Simulate processing time
    encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    qdrant=vector_database_init()
    llm_output = f"The analysis indicates a possible condition based on the symptoms: {vector_search(qdrant,encoder,symptoms,"Clinical")}"
    return llm_output


def generate_random_eta():
    random_seconds = random.randint(60, 7200)
    eta = datetime.timedelta(seconds=random_seconds)
    total_seconds = int(eta.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    formatted_eta = f"{hours:02}:{minutes:02}:{seconds:02}"
    return formatted_eta

In [3]:

ref_model,nlp=model_init()

print("Hello! My name is RIYA and i work as a receptionist for Dr. Adrin. How can I help you today?")
while True:
    print("\nRIYA : Please  Send a message or Report an emergency")
    user_choice = input("RIYA : Please  Send a message or Report an emergency")
    choice=process_response(user_choice)
    if choice=="Message":
        message = input("\nPlease enter the message you want to send: ")
        print( "RIYA :Thanks for the message, we will forward it to Dr. Adrin")
        break
    elif choice == "Emergency":
        print("\nRIYA : You selected 'Report an emergency'.")
        symptoms = input("Please describe the symptoms: ")
        print(f"User: {symptoms}")
        llm_output = []
        def process_and_store():
            result = process(symptoms)
            llm_output.append(result)  # Store the semantic similarity search output in a list
        processing_thread = threading.Thread(target=process_and_store)
        processing_thread.start()

        print("RIYA : Please hold just a sec")    
            # Simultaneously ask for location
        location = input("RIYA : I am checking what you should do immediately, meanwhile, can you tell me which area are you located right now?")
        ETA=generate_random_eta()
        print(f"User: {location}")
        if location_ner(nlp,location)!=0:
            print(f"RIYA : Location '{location}' received.  Dr. Adrin will be coming to their location immediately. ETA:{ETA}")
        else: 
            location=input("I don't understand that and could you please repeat the area you are located right now (in proper format)? ")
            print(f"User: {location}")
            if(location_ner(nlp,location)==0):
                print("RIYA : I don’t understand that and unable to fetch your location ,could you please try again later .")
                exit()
            else:
                print(f"RIYA : Location '{location}' received.  Dr. Adrin will be coming to their location immediately. ETA:{ETA}")
        processing_thread.join()
            
            # Print the semantic similarity search output after location input
        print("\nRIYA :  Processing complete. Here is the analysis from our system:")
        print(llm_output[0])  # Access the first (and only) element in the list
        response = input("Do you have anything else to ask ?") 
        print(f"User: {response}")
        res_rev=process_response(response)
        #res_rev=relevance_check(ref_model,response)
        if res_rev=="Late":
            print(f"RIYA : I understand that you are worried that Dr. Adrin will arrive too late, meanwhile we would suggest that you follow following remedy:{llm_output[0]} \n"  )

        else:
            print("RIYA : I don't understand that but Don’t worry, please follow the steps, Dr. Adrin will be with you shortly")
        break
    else:
        print("\nRIYA : Invalid Response. Please mention wether you like to send a message or to report an emergency.")
    
print("\nRIYA : Thank you for using the service. Stay safe!")



e:\conda\envs\GEN_AI_T1\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Hello! My name is RIYA and i work as a receptionist for Dr. Adrin. How can I help you today?

RIYA : Please  Send a message or Report an emergency

RIYA : You selected 'Report an emergency'.
User: I am not able to breathe properly dealing with chest pain

Processing the emergency report...
RIYA : Please hold just a sec
User: Indore MP India
RIYA : Location 'Indore MP India' received.  Dr. Adrin will be coming to their location immediately. ETA:00:23:44

RIYA :  Processing complete. Here is the analysis from our system:
The analysis indicates a possible condition based on the symptoms: ({'name': 'Unconscious', 'remedy': 'If someone is unconscious or unresponsive, follow the ABC principle: Airway, Breathing, and Circulation. \n               1. Airway: Open their airway if they are not breathing. \n               2. Breathing: If the airway is clear but the person is still not breathing, provide rescue breathing.\n               3. Circulation: Perform chest compressions to maintain bloo